In [2]:
# Importa o findspark e inicializa
import findspark
findspark.init()

# Import required modules
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, DateType, TimestampType, ArrayType
from pyspark.sql.functions import col, sum, from_json, unix_timestamp, window
import pyspark.sql.functions as F

In [3]:
# Cria a sessão Spark
spark = SparkSession.builder.appName("case-improving").getOrCreate()

24/05/31 17:53:19 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/05/31 17:53:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 17:53:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/31 17:53:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/31 17:53:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/31 17:53:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
from pyspark.sql.types import StringType, MapType
mapCol = MapType(StringType(),StringType(),False)

In [14]:
from pyspark.sql.types import StructField, StructType, StringType, MapType
schema = StructType([
    StructField('name', StringType(), True),
    StructField('properties', ArrayType(MapType(StringType(),StringType()),True))
])

In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
dataDictionary = [
        # ('James',[{'hair':'black','eye':'brown'}]),
        # ('Michael',[{'hair':'brown','eye':None}]),
        ('Robert',[{'hair':'red','eye':'black'}]),
        ('Washington',[{'hair':'grey','eye':'grey'}]),
        ('Jefferson',[{'hair':'brown','eye':''}])
        ]
df = spark.createDataFrame(data=dataDictionary, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|Robert    |[{eye -> black, hair -> red}]|
|Washington|[{eye -> grey, hair -> grey}]|
|Jefferson |[{eye -> , hair -> brown}]   |
+----------+-----------------------------+



In [67]:
import json
output_json = []

for row in df.rdd.toLocalIterator():
    print(row.properties)
    for item in row.properties:
        print(item)
        output_json.append((item))

print(output_json)


output_json_df = spark.createDataFrame(data=output_json)
output_json_df.show()


[{'eye': 'black', 'hair': 'red'}]
{'eye': 'black', 'hair': 'red'}
[{'eye': 'grey', 'hair': 'grey'}]
{'eye': 'grey', 'hair': 'grey'}
[{'eye': '', 'hair': 'brown'}]
{'eye': '', 'hair': 'brown'}
[{'eye': 'black', 'hair': 'red'}, {'eye': 'grey', 'hair': 'grey'}, {'eye': '', 'hair': 'brown'}]
+-----+-----+
|  eye| hair|
+-----+-----+
|black|  red|
| grey| grey|
|     |brown|
+-----+-----+



In [8]:
from pyspark.sql.functions import explode
df.select(df.name,explode(df.properties)).show()

+----------+----+-----+
|      name| key|value|
+----------+----+-----+
|     James| eye|brown|
|     James|hair|black|
|   Michael| eye| NULL|
|   Michael|hair|brown|
|    Robert| eye|black|
|    Robert|hair|  red|
|Washington| eye| grey|
|Washington|hair| grey|
| Jefferson| eye|     |
| Jefferson|hair|brown|
+----------+----+-----+



In [9]:
from pyspark.sql.functions import map_keys
df.select(df.name,map_keys(df.properties)).show()

+----------+--------------------+
|      name|map_keys(properties)|
+----------+--------------------+
|     James|         [eye, hair]|
|   Michael|         [eye, hair]|
|    Robert|         [eye, hair]|
|Washington|         [eye, hair]|
| Jefferson|         [eye, hair]|
+----------+--------------------+



In [10]:
from pyspark.sql.functions import explode,map_keys
keysDF = df.select(explode(map_keys(df.properties))).distinct()
keysList = keysDF.rdd.map(lambda x:x[0]).collect()
print(keysList)
#['eye', 'hair']

['eye', 'hair']


In [12]:
from pyspark.sql.functions import map_values
df.select(df.name,map_values(df.properties)).show()

+----------+----------------------+
|      name|map_values(properties)|
+----------+----------------------+
|     James|        [brown, black]|
|   Michael|         [NULL, brown]|
|    Robert|          [black, red]|
|Washington|          [grey, grey]|
| Jefferson|             [, brown]|
+----------+----------------------+

